## Greedy-FMT train on ACS-PUMS

In [ ]:
class MTL(nn.Module):

    def __init__(self,d_in=50,tasks=2):
        super(MTL, self).__init__()
        self.tasks=tasks
        self.fc1 = nn.Linear(d_in, 1024)  
        self.bn1= nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 128)
        
        self.bn3 = nn.BatchNorm1d(1024)
        self.fc4 = nn.Linear(1024,1024)
        self.tasks_out=nn.ModuleDict({str(t):nn.Linear(128,2) for t in range(self.tasks)})

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        
        x = F.relu(self.bn3(self.fc4(x)))
        
        x = F.relu(self.bn2(self.fc2(x)))        
        x = F.relu(self.fc3(x))

        t=[self.tasks_out[str(i)](x) for i in range(self.tasks)]
        
        return t


In [26]:
def fair_loss(output,target,x_control):
    prot_att=x_control
    index_prot=torch.squeeze(torch.nonzero(prot_att[:] != 1.))
    target_prot=torch.index_select(target, 0, index=index_prot)
    index_prot_pos=torch.squeeze(torch.nonzero(target_prot[:] == 1. ))
    index_prot_neg=torch.squeeze(torch.nonzero(target_prot[:] == 0. ))

    index_non_prot=torch.squeeze(torch.nonzero(prot_att[:] == 1.))
    target_non_prot=torch.index_select(target, 0, index=index_non_prot)
    index_non_prot_pos=torch.squeeze(torch.nonzero(target_non_prot[:] == 1. ))
    index_non_prot_neg=torch.squeeze(torch.nonzero(target_non_prot[:] == 0. ))

    l_prot_pos=F.cross_entropy(torch.index_select(output, 0, index=index_prot_pos),torch.index_select(target, 0, index=index_prot_pos))    
    l_non_prot_pos=F.cross_entropy(torch.index_select(output, 0, index=index_non_prot_pos),torch.index_select(target, 0, index=index_non_prot_pos))    
    l_non_prot_neg=F.cross_entropy(torch.index_select(output, 0, index=index_non_prot_neg),torch.index_select(target, 0, index=index_non_prot_neg))
    l_prot_neg=F.cross_entropy(torch.index_select(output, 0, index=index_prot_neg),torch.index_select(target, 0, index=index_prot_neg))    

    for l in [l_prot_pos,l_non_prot_pos,l_prot_neg,l_non_prot_neg]:
        if torch.isinf(l)==True:
            l=torch.zeros_like(l,requires_grad=True)
    dl_pos=torch.max(l_prot_pos,l_non_prot_pos)
    dl_neg=torch.max(l_prot_neg,l_non_prot_neg)
    L=dl_pos+dl_neg
    
    return L

In [31]:
import torchmetrics
acc = torchmetrics.Accuracy()
def DM_rate(output,target,x_control):
    prot_att=x_control
    index_prot=torch.squeeze(torch.nonzero(prot_att[:] != 1.))
    target_prot=torch.index_select(target, 0, index=index_prot)
    index_prot_pos=torch.squeeze(torch.nonzero(target_prot[:] == 1. ))
    index_prot_neg=torch.squeeze(torch.nonzero(target_prot[:] == 0. ))

    index_non_prot=torch.squeeze(torch.nonzero(prot_att[:] == 1.))
    target_non_prot=torch.index_select(target, 0, index=index_non_prot)
    index_non_prot_pos=torch.squeeze(torch.nonzero(target_non_prot[:] == 1. ))
    index_non_prot_neg=torch.squeeze(torch.nonzero(target_non_prot[:] == 0. ))

    if index_prot_pos.shape==torch.Size([]) or index_prot_pos.shape==torch.Size([0])\
        or index_non_prot_pos.shape==torch.Size([]) or index_non_prot_pos.shape==torch.Size([0]):
            l_prot_pos=torch.tensor(0.0001)
            l_non_prot_pos=torch.tensor(0.0001)
    else:        
            l_prot_pos=acc(torch.index_select(output, 0, index=index_prot_pos),torch.index_select(target, 0, index=index_prot_pos))    
            l_non_prot_pos=acc(torch.index_select(output, 0, index=index_non_prot_pos),torch.index_select(target, 0, index=index_non_prot_pos))    
    
    if index_prot_neg.shape==torch.Size([]) or index_prot_neg.shape==torch.Size([0])\
        or index_non_prot_neg.shape==torch.Size([]) or index_non_prot_neg.shape==torch.Size([0]):
            l_prot_neg=torch.tensor(0.0001)
            l_non_prot_neg=torch.tensor(0.0001)
    else:        
            l_prot_neg=acc(torch.index_select(output, 0, index=index_prot_neg),torch.index_select(target, 0, index=index_prot_neg))    
            l_non_prot_neg=acc(torch.index_select(output, 0, index=index_non_prot_neg),torch.index_select(target, 0, index=index_non_prot_neg))  
            
    dl_pos=torch.abs(l_prot_pos-l_non_prot_pos)
    dl_neg=torch.abs(l_prot_neg-l_non_prot_neg)
    DM=dl_pos+dl_neg
    
    return DM

In [ ]:
net=MTL(model_pre.to(dv),tasks=N_tasks)
net=nn.DataParallel(net)
net.to(dv)
opti_S=optim.AdamW(params=net.parameters())

In [ ]:
omega_S=torch.tensor([1/N_tasks for t in range(N_tasks)])
init_loss_S=[None for t in range(N_tasks)]
m_acc,flag=0.0,1
criteria = nn.CrossEntropyLoss()
loss_pointer=[1]
best_S=[[0,0] for t in range(N_tasks)]
optim_path,optim_path_disc,optim_clas=[],[],[]#[[] for p in x_control]
path='/yourpath/model/Vanilla_MTL.pt'
ML_AF_GN=[[] for t in range(N_tasks)]
torch.save(net.state_dict(),path)
#net.load_state_dict(torch.load(path))
for epoch in range(50):  # loop over each NN multiple times
    i,batch=0,8192
    j=0
    while(i<len(X_train)):
        # get the inputs; data is a list of [inputs, labels]
        if (i+batch)<len(X_train):
            inputs, in_t = torch.tensor(X_train[i:i+batch]),in_tr[i:i+batch]
            labels=[y_train[t][i:i+batch] for t in range(N_tasks)]
            #if epoch<pretrn:
            xc=xg[i:i+batch]
            i=i+batch 
        else:
            inputs,in_t = torch.tensor(X_train[i:]),in_tr[i:]
            labels=[y_train[t][i:] for t in range(N_tasks)]
            #if epoch<pretrn:
            xc=xg[i:]
            i=len(X_train)
        
        # zero the parameter gradients
        #net.load_state_dict(torch.load(path))           
        opti_S.zero_grad()
        outputs = net(inputs.to(dv).float())
        lp,grads,G_n,loss_ratio=[0 for t in range(N_tasks)],[],[],[]
        for t in range(N_tasks):
            loss_a=criteria(outputs[t], labels[t].to(dv))
            loss_f=fair_loss(outputs[t], labels[t].to(dv),xc.to(dv))
            if loss_a>loss_f:
                loss_t=loss_a
            else:
                loss_t=loss_f
            loss_t.backward(retain_graph=True)
            
            if init_loss_S[t]== None:
                init_loss_S[t]=loss_t.item()
            loss_ratio.append(loss_t.item()/init_loss_S[t])
            grads_sh={}
            for n,p in net.named_parameters():
                if p.data.shape[0]!=2 and p.grad!=None:
                    grads_sh[n] = p.grad
                    p.grad=None
                    #p.grad.zero_()
            grads.append(grads_sh)
            G_n.append(torch.linalg.norm(torch.stack([torch.linalg.norm(grads_sh[g]) for g in grads_sh])))
        G_n = torch.stack(G_n)
        #print('GN:',G_n)
        E_t = sum(loss_ratio)/len(loss_ratio)
        r_t=[loss/E_t for loss in loss_ratio] #relative inverse trainin g rate of student
        omega_S,net = Update_model(net,grads,omega_S,G_n,r_t,opti_S) 
        torch.save(net.state_dict(),path)
        
    with torch.no_grad():
        pred0=net(torch.tensor(X_val).to(device).float())
        print('Epoch: ',epoch)
        for t in range(N_tasks):            
            accuracy=acc(pred0[t],y_v[t])
            EO=DM_rate(pred0[t],torch.tensor(y_v[t]),torch.tensor(g_val))
            ML_AF_GN[t].append([accuracy,EO])
            np.save('/home/roy/model/PUMS_GN_AF.npy',ML_AF_GN)
            print('Task',t,'  Acc:',accuracy, 'EO:',EO)#'Loss pointers',loss_pointer[-3:])
            if accuracy>best_S[t][0]:
                best_S[t][0]=accuracy
            if 1-EO>best_S[t][1]:
                best_S[t][1]=1-EO

## Greedy-FMT train on CelebA

In [15]:
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import torchvision.models as pretrain
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [20]:
model_pre=pretrain.resnet18(pretrained=True)

In [28]:
for par in model_pre.parameters():
    par.requires_grad=False
in_fc=model_pre.fc.in_features
model_pre.fc=nn.Linear(in_fc,1024)

In [30]:
class MTL(nn.Module):

    def __init__(self,pretrain,tasks=2):
        super(MTL, self).__init__()
        self.tasks=tasks
        self.fc1 = pretrain
        self.bn1= nn.BatchNorm1d(1024)
        self.fc2 = nn.Linear(1024, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.fc3 = nn.Linear(512, 128)
        
        self.bn3 = nn.BatchNorm1d(1024)
        self.fc4 = nn.Linear(1024,1024)
        self.tasks_out=nn.ModuleDict({str(t):nn.Linear(128,2) for t in range(self.tasks)})

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        
        x = F.relu(self.bn3(self.fc4(x)))
        
        x = F.relu(self.bn2(self.fc2(x)))        
        x = F.relu(self.fc3(x))

        t=[self.tasks_out[str(i)](x) for i in range(self.tasks)]
        
        return t

In [43]:
lr=0.001
def Update_model(model,grads_sh,omega,G_n,r_t,opti):
    loss_gn=[(G_n[t]-torch.mean(G_n)*r_t[t]) for t in range(len(G_n))]
    for i in range(len(G_n)):
        d_l=0
        if loss_gn[i]>0:
            d_l+=(len(G_n)-1)/len(G_n)*G_n[i]
        elif loss_gn[i]<0:
            d_l-=(len(G_n)-1)/len(G_n)*G_n[i]
        for j in range(len(G_n)):
            if j!=i:
                if loss_gn[j]>0:
                    d_l-=(G_n[i]/len(G_n))
                elif loss_gn[j]<0:
                    d_l+=(G_n[i]/len(G_n))
        
        omega[i]-=lr*d_l
    for t in range(len(G_n)):
        for n,p in model.named_parameters():
            if p.data.shape[0]!=2 and p.grad!=None:
                if t==0:
                    p.grad=omega[t]*grads_sh[t][n]
                else:
                    p.grad+=omega[t]*grads_sh[t][n]
                    
    opti.step() 
    total=sum(omega)
    for i in range(len(omega)):
        omega[i]=omega[i]/total
    return omega,model

In [44]:
net=MTL(model_pre.to(dv),tasks=N_tasks)
net=nn.DataParallel(net)
net.to(dv)
opti_S=optim.AdamW(params=net.parameters())

In [ ]:
omega_S=torch.tensor([1/N_tasks for t in range(N_tasks)]).to(dv)
init_loss_S=[None for t in range(N_tasks)]
m_acc,flag=0.0,1
criteria = nn.CrossEntropyLoss()
optim_path,optim_path_disc,optim_clas=[],[],[]#[[] for p in x_control]
path='/yourpath/CelebA/model/Vanilla_MTL.pt'


torch.save(net.state_dict(),path)


dt=[dataloader1,dataloader2,dataloader3,dataloader4]
yt=[y_tr1,y_tr2,y_tr3,y_tr4]
gt=[g_tr1,g_tr2,g_tr3,g_tr4]
for epoch in range(20):  # loop over each NN multiple times
    loss_pointer=[]
    state_net,state_a,state_f=[[] for t in range(N_tasks)],[[] for t in range(N_tasks)],[[] for t in range(N_tasks)]
    R_A,R_F=[[] for t in range(N_tasks)],[[] for t in range(N_tasks)]
    ch=np.random.choice([0,1,2,3])
    dataloader,y_train,g_train=dt[ch],yt[ch],gt[ch]
    y_train=[torch.tensor(y_train[i]) for i in range(N_tasks)]
    xg=torch.tensor(g_train)
    for i, data in enumerate(dataloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs =data[0]
        if ((i+1)*batch_size)<len(in_tr):            
            labels=[y_train[t][i*batch_size:(i+1)*batch_size] for t in range(N_tasks)]
            xc=xg[i*batch_size:(i+1)*batch_size]
        else:
            labels=[y_train[t][i*batch_size:] for t in range(N_tasks)]
            xc=xg[i*batch_size:]
        
        # zero the parameter gradients
        net.load_state_dict(torch.load(path))
        net.to(dv)
                       
        opti_S.zero_grad()
        outputs = net(inputs.to(dv).float())
        grads,G_n,loss_ratio=[],[],[]
        for t in range(N_tasks):
            
            #MTL starts
            loss_a=criteria(outputs[t], labels[t].to(dv))
            loss_f=fair_loss(outputs[t], labels[t].to(dv),xc.to(dv))
            if loss_a>loss_f:
                loss_t=loss_a
            else:
                loss_t=loss_f
            loss_t.backward(retain_graph=True)
            
            if init_loss_S[t]== None:
                init_loss_S[t]=loss_t.item()
            loss_ratio.append(loss_t.item()/init_loss_S[t])
            grads_sh={}
            for n,p in net.named_parameters():
                if p.data.shape[0]!=2 and p.grad!=None:
                    grads_sh[n] = p.grad
                    p.grad=None
                    #p.grad.zero_()
            grads.append(grads_sh)
            G_n.append(torch.linalg.norm(torch.stack([torch.linalg.norm(grads_sh[g]) for g in grads_sh])))
        G_n = torch.stack(G_n)
        E_t = sum(loss_ratio)/len(loss_ratio)
        r_t=[loss/E_t for loss in loss_ratio] #relative inverse trainin g rate of student
        omega_S,net = Update_model(net,grads,omega_S,G_n,r_t,opti_S) 
        torch.save(net.state_dict(),path)
        torch.save(omega_S,'/yourpath/CelebA/model/omega.pt')
  
    torch.cuda.empty_cache()
    with torch.no_grad():
        #Val_mod.load_state_dict(torch.load(path))
        
        ch=np.random.choice([0,1])
        val_batch,y_v,g_val=next(iter(d_v[ch]))[0],yv[ch],gv[ch]
        pred0=net(val_batch.to(dv))#=Val_mod
        print('Epoch: ',epoch)
        for t in range(N_tasks): 
            accuracy=acc(pred0[t].to(cpu),torch.tensor(y_v[t]))
            EO=DM_rate(pred0[t].to(cpu),torch.tensor(y_v[t]),torch.tensor(g_val))
            
            print('Task',t,'  Acc:',accuracy, 'EO:',EO)
        